In [7]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

esp_ff = foyer.Forcefield(forcefield_files="/Users/madilyn/Projects/repos/forcefields/CPDT.xml")

# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
CPDT = mb.load("/Users/madilyn/Projects/repos/forcefields/CPDT_typed.mol2")

for p in CPDT.particles():
    p.name = f"_{p.name}"
    
box = mb.fill_box(compound=CPDT, n_compounds=5, box=[10,10,10])
box_pmd = esp_ff.apply(box)

snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _S1
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C2
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C1
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H3
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H2
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [8]:
#setting cpu and simulation 
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

#setting the integrator
kt = 1.2
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt = 0.01)
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=1.2, tau=0.5) #what is tau?
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_CPDT.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)

In [9]:
sim.run(1e4)

*Warning*: charge.pppm: system is not neutral and unscreened interactions are calculated, the net charge is -47.4739
*Warning*: charge.pppm: RMS error of 0.311306 is probably too high! 0.311299 0.311306
**ERROR**: Particle with unique tag 50 is no longer in the simulation box.

Cartesian coordinates: 
x: -1972.13 y: -1810.71 z: 52.8239
Fractional coordinates: 
f.x: -69.7788 f.y: -64.0264 f.z: 2.38243
Local box lo: (-14.0308, -14.0308, -14.0308)
          hi: (14.0308, 14.0308, 14.0308)


RuntimeError: Error computing cell list